In [2]:
from src.datasets.snli import *
from src.model.snli_models import *
from src.utils.snli_utils import *

In [3]:
snli_conf = {"batch_size":128,"max_len":40,"device":'cuda',"tokenizer":'spacy'}
dataset = SNLIDataModule(snli_conf)

In [4]:
dataset.prepare_data()

In [5]:
model_conf = {
    "hidden_size":300,
    "embedding_dim":300,
    "dropout":0.3,
    "use_glove":True,
    "num_layers":1,
    "dataset":"snli",
    "fcs":1,
    "vocab_size":dataset.vocab_size(),
    "tokenizer":"spacy",
    "padding_idx":dataset.padding_idx(),
    "attention_layer_param":200
}

hparams = {
    "optimizer_base": {
        "optim": "adamw",
        "lr": 0.0010039910781394373,
        "scheduler": "const",
    },
    "optimizer_tune": {
        "optim": "adam",
        "lr": 0.0010039910781394373,
        "weight_decay": 0.1,
        "scheduler": "lambda",
    },
    "switch_epoch": 5,
}


model = SNLI_model(attn_bilstm_snli,model_conf,hparams=hparams)

In [6]:
EPOCHS = 20

tensorboard_logger = TensorBoardLogger("lightning_logs")
lr_logger = LearningRateLogger(logging_interval="step")

trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS, 
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger, SwitchOptim()],
    logger=[tensorboard_logger],
    row_log_interval=2,
) 
trainer.fit(model, dataset)
trainer.test(model, datamodule=dataset)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | BiLSTM_snli | 14 M  


In [10]:
for i in dataset.train_dataloader():
    print(i)
    break


[torchtext.data.batch.Batch of size 128 from SNLI]
	[.premise]:[torch.cuda.LongTensor of size 128x40 (GPU 0)]
	[.hypothesis]:[torch.cuda.LongTensor of size 128x40 (GPU 0)]
	[.label]:[torch.cuda.LongTensor of size 128 (GPU 0)]


In [9]:
i.label

tensor([0, 1, 2, 0, 0, 2, 2, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1,
        0, 1, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 1, 1, 1,
        0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 0, 0, 2,
        0, 2, 0, 1, 1, 2, 1, 1, 0, 2, 1, 0, 2, 2, 0, 0, 1, 0, 0, 0, 1, 2, 0, 2,
        2, 0, 0, 1, 2, 0, 2, 0, 1, 1, 1, 1, 0, 2, 1, 2, 0, 1, 1, 2, 0, 0, 2, 1,
        0, 2, 1, 0, 1, 0, 0, 1], device='cuda:0', dtype=torch.int32)

In [11]:
opt = model(i.premise,i.hypothesis)

In [15]:
opt = opt.squeeze(0)

In [16]:
import torch.nn.functional as F
F.cross_entropy(opt,i.label)

RuntimeError: Expected object of scalar type Long but got scalar type Int for argument #2 'target' in call to _thnn_nll_loss_forward